### Pavan Pyla
### 22232
### ASSIGNMENT -2 Fashion MNISAT Data training and validation



In [1]:

import torch.nn as nn                           # Importing the Neccesary Libraries
!pip install torch torchvision

In [3]:
import torch
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

ds_train = datasets.FashionMNIST(
    root = 'data',                                          # Downloadint the Training Data set from Inbuilt Datasets
    train = True,
    download = True,
    transform = ToTensor(),                                # Converting image to tensor
    target_transform= Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(0,torch.tensor(y), value=1))

)

test_dataset = datasets.FashionMNIST(
    root = 'data',
    train = False,
    download = True,                         # Downloading test data
    transform = ToTensor(),
    target_transform= Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(0,torch.tensor(y), value=1))

)


100%|██████████| 26421880/26421880 [00:01<00:00, 19158760.36it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 358231.81it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 6208787.11it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 16751184.63it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [4]:


from torch.utils.data import random_split

train_size = len(ds_train)                        # Getting the validation set from training data which is 10% of training data
val_size = int(0.1 * train_size)

train_dataset, val_dataset = random_split(ds_train, [train_size - val_size, val_size])

print(f"Training set size: {len(train_dataset)}")
print(f"Validation set size: {len(val_dataset)}")

Training set size: 54000
Validation set size: 6000


In [5]:

print(f"Test set size: {len(test_dataset)}")     # gettting the len of test dataset

Test set size: 10000


### Simple Neural Network

In [7]:

class SimpleNet(nn.Module):              # building a Simple Neural Network
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.linear1 = nn.Linear(in_features=28*28, out_features=128)  # Input size 784 (28x28), output size 128
        self.linear2 = nn.Linear(in_features=128, out_features=64)   # Input size 128, output size 64
        self.linear3 = nn.Linear(in_features=64, out_features=10)    # Input size 64, output size 10 for 10 classes

    def forward(self, x): # forward prop function
        x = x.view(x.size(0),-1)
        x = torch.relu(self.linear1(x))
        x = torch.relu(self.linear2(x))
        x = torch.softmax(self.linear3(x),dim = 1)
        return x


simple_net = SimpleNet()   # instance or object creation


In [8]:

import torch.optim as optim

In [9]:
criterion = nn.CrossEntropyLoss()  # Taking Cross Entropy Loss as the Loss function
optimizer = optim.SGD(simple_net.parameters(), lr=0.01)  # optimizer is SGD stochastic gradient descent

In [10]:
from torch.utils.data import DataLoader # importing dataloader

### Training and Validation Function

In [12]:
# loading all the datasets into dataloader

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)             # batch size is 64
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [13]:
def trainval_loop(train_loader, val_loader, simple_net, criterion, optimizer):

    simple_net.train()


    size = len(train_loader.dataset)
    for batch, (X, y) in enumerate(train_loader):

        pred = simple_net(X)
        loss = criterion(pred, y)

                                                 # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        if batch % 100 == 0:
            current = batch * len(X)
            print(f"Training loss: {loss.item():>7f}  [{current:>5d}/{size:>5d}]") #printing training loss


    simple_net.eval()

    # Validation loop
    val_loss = 0
    val_size = len(val_loader.dataset)
    with torch.no_grad():
        for batch, (X, y) in enumerate(val_loader):
            # Compute prediction and loss
            pred = simple_net(X)
            loss = criterion(pred, y)
            val_loss += loss.item()

            if batch % 100 == 0:
                current = batch * len(X)
                print(f"Validation loss: {loss.item():>7f}  [{current:>5d}/{val_size:>5d}]") #printing validation loss

    avg_val_loss = val_loss / len(val_loader)
    print(f"Avg. Validation loss: {avg_val_loss:>7f}")            #printing average validation loss



### Tessting Function

In [15]:

def test_loop(test_loader, simple_net, criterion):
    simple_net.eval()
    size = len(test_loader.dataset)
    num_batches = len(test_loader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in test_loader:
            pred = simple_net(X)
            test_loss += criterion(pred, y).item()                     # testing loop for accuracy check , basically tessting the model


            pred_labels = pred.argmax(dim=1)
            y= y.argmax(dim=1)

            correct += (pred_labels == y).sum().item()

    test_loss /= num_batches
    accuracy = correct / size
    print(f"Test Error: \n Accuracy: {(100*accuracy):.2f}%, Avg loss: {test_loss:.8f} \n")




### Calling the Train and validation function

In [18]:
epochs = 10
for t in range(epochs):
    print(f'===EPOCH===  {t}')
    trainval_loop(train_loader,val_loader, simple_net, criterion, optimizer)      # running train_val loop by 10 times



===EPOCH===  0
Training loss: 1.853493  [    0/54000]
Training loss: 1.842747  [ 6400/54000]
Training loss: 1.919159  [12800/54000]
Training loss: 1.851718  [19200/54000]
Training loss: 1.792981  [25600/54000]
Training loss: 1.822859  [32000/54000]
Training loss: 1.845055  [38400/54000]
Training loss: 1.794331  [44800/54000]
Training loss: 1.866264  [51200/54000]
Validation loss: 1.788117  [    0/ 6000]
Avg. Validation loss: 1.801836
===EPOCH===  1
Training loss: 1.858188  [    0/54000]
Training loss: 1.791188  [ 6400/54000]
Training loss: 1.794666  [12800/54000]
Training loss: 1.815450  [19200/54000]
Training loss: 1.765817  [25600/54000]
Training loss: 1.790637  [32000/54000]
Training loss: 1.760355  [38400/54000]
Training loss: 1.702230  [44800/54000]
Training loss: 1.733289  [51200/54000]
Validation loss: 1.778180  [    0/ 6000]
Avg. Validation loss: 1.790135
===EPOCH===  2
Training loss: 1.823246  [    0/54000]
Training loss: 1.825524  [ 6400/54000]
Training loss: 1.720633  [12800

In [19]:
test_loop(test_loader, simple_net, criterion) #test_loop running and checking the results

Test Error: 
 Accuracy: 78.42%, Avg loss: 1.68926626 

